In [4]:
import pandas as pd

df_path = '/home/rinaen/PycharmProjects/communication_agent/testing/eval_chunks.csv'
df = pd.read_csv(df_path)

In [5]:
from src.data_acquisition.content_processing.content_classification import get_content_type_simple


def get_classified(chunks: list[str]):
    classified = []
    for chunk in chunks:
        try:
            classified.append(get_content_type_simple(chunk))
        except Exception:
            classified.append(None)
    return classified

In [6]:
chunks = df['chunk'].tolist()
classified = get_classified(chunks)
df['zero-shot'] = classified
display(df)

,url,chunk,type,classified
0,https://www.gotobrno.cz/wp-content/uploads/202...,Element\n“An excellent fusion of top-quality m...,place,place
1,https://www.gotobrno.cz/wp-content/uploads/202...,Hostinec U Semináru\n“In the mood for a Czech ...,place,place
2,https://www.gotobrno.cz/wp-content/uploads/202...,Kohout NA VÍNĚ\n“Ideal for festive occasions t...,place,place
3,https://www.gotobrno.cz/wp-content/uploads/202...,MANYA Sushi & Sake Izakaya\n“Think you know ev...,place,place
4,https://www.gotobrno.cz/wp-content/uploads/202...,Mikrofarma\n“You can tell these guys know meat...,place,place
...,...,...,...,...
95,https://www.ndbrno.cz/en/program/,Anton Pavlovich Chekhov\nThree Sisters\nMahen ...,event,event
96,https://www.mzm.cz/intervence-premysl-hytych-/...,INTERVENTION. Přemysl Hytych / Floral object\n...,event,event
97,https://www.mzm.cz/mista/pripravovane-vystavy,Opening of the exhibition Thousand Faces of Am...,event,event
98,https://moravska-galerie.cz/vystavy/stale-expo...,Admission fee voluntary\nART DESIGN FASHION\nM...,event,event


In [7]:
# rename column 'classified' to zero-shot
df.rename(columns={'classified': 'zero-shot'}, inplace=True)


In [10]:
same_df = df.copy()
same_df['same'] = df['zero-shot'] == df['type']
same = same_df['same'].value_counts()
print(f"Correctly classified: {same[True]} %")

Correctly classified: 46 %


In [25]:
place_df = same_df[same_df['type'] == 'place']
correct_places = place_df['same'].sum()
places_percentage = correct_places / len(place_df) * 100
print(
    f"Correctly classified places: {correct_places} out of {len(place_df)}, which makes {places_percentage} %")


Correctly classified places: 25 out of 25, which makes 100.0 %


In [26]:
static_df = same_df[same_df['type'] == 'static']
correct_static = static_df['same'].sum()
static_percentage = correct_static / len(static_df) * 100
print(
    f"Correctly classified static: {correct_static} out of {len(static_df)}, which makes {static_percentage} %")

Correctly classified static: 0 out of 25, which makes 0.0 %


In [27]:
administration_df = same_df[same_df['type'] == 'administration']
correct_administration = administration_df['same'].sum()
administration_percentage = correct_administration / len(administration_df) * 100
print(
    f"Correctly classified administration: {correct_administration} out of {len(administration_df)}, which makes {administration_percentage} %")

Correctly classified administration: 1 out of 25, which makes 4.0 %


In [28]:
event_df = same_df[same_df['type'] == 'event']
correct_event = event_df['same'].sum()
event_percentage = correct_event / len(event_df) * 100
print(
    f"Correctly classified event: {correct_event} out of {len(event_df)}, which makes {event_percentage} %")

Correctly classified event: 20 out of 25, which makes 80.0 %


In [31]:
eval_df = pd.DataFrame({
    'Třída': ['place', 'static', 'administration', 'event', 'všechny'],
    'Správně': [correct_places, correct_static, correct_administration, correct_event, same[True]],
    'Celkem': [len(place_df), len(static_df), len(administration_df), len(event_df), len(same_df)],
    '[%]': [places_percentage, static_percentage, administration_percentage, event_percentage,
            same[True] / len(same_df) * 100]
})
display(eval_df)

,Třída,Správně,Celkem,[%]
0,place,25,25,100.0
1,static,0,25,0.0
2,administration,1,25,4.0
3,event,20,25,80.0
4,všechny,46,100,46.0


In [32]:
eval_df.to_csv('zero_shot_eval.csv', index=False)